In [ ]:
pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# **Preprocessing Datasets**

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Step 1: Prepare the data
# Function to read a text file and return its content as a list of lines
# def read_text_file(file_path):
#     with open(file_path, 'r', encoding='utf-8') as file:
#         content = file.readlines()
#     return content

# # Example usage:

# conversational_data = read_text_file('conversational_dataset.txt')

# # Vocabulary dataset
# words = [line.strip() for line in vocabulary_data]

# # Conversational dataset
# # Separate questions and answers
# questions = conversational_data[0::2]
# answers = conversational_data[1::2]
# print(questions)
# print(answers)
#   # Add start and end tokens to the answers



def read_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.readlines()
    return content

vocabulary_data = read_text_file('বাংলা_শব্দ_তালিকা.txt')
conversational_data = read_text_file('conversational_dataset1.txt')

words = [line.strip() for line in vocabulary_data]

questions = []
answers = []

for i in range(0, len(conversational_data), 3):  # Skipping every third line (the empty line)
    question_line = conversational_data[i].strip()
    answer_line = conversational_data[i + 1].strip()

    # # Check if the line is a valid question-answer pair
    # if question_line.endswith('?') and answer_line.endswith('।'):
    #     questions.append(question_line)
    #     answers.append("\t " + answer_line + " \n")

    questions.append(question_line)
    answers.append("\t " + answer_line + " \n")
# print(questions)
# print(answers)

questions = [q.strip() for q in questions]
answers = ["\t " + a.strip() + " \n" for a in answers]





# Step 2: Tokenize the words using the vocabulary dataset
# # Add the start token explicitly to your vocabulary list
# words.append('\t')
# tokenizer = Tokenizer(char_level=True)  # Char-level tokenization for languages like Bengali
# tokenizer.fit_on_texts(words)

# # Check if the start token index is now valid
# start_token_index = tokenizer.word_index.get('\t', None)
# if start_token_index is None or start_token_index >= vocab_size:
#     raise ValueError("Start token index is still invalid. Consider increasing vocab_size.")

# Fit tokenizer with your dataset
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(words + ['\t'])  # Including the start token

# Determine the new vocab_size
vocab_size = len(tokenizer.word_index) + 1  # +1 for padding token
print("New vocab_size:", vocab_size)


# # Step 2: Tokenize the words using the vocabulary dataset
# tokenizer = Tokenizer(char_level=True)  # Char-level tokenization for languages like Bengali
# tokenizer.fit_on_texts(words)

# # Step 2.1: Re-index the tokenizer (if necessary)
# original_word_index = tokenizer.word_index
# new_word_index = {}
# new_index = 1  # Start from 1 since 0 is usually reserved for padding

# for word, index in original_word_index.items():
#     if index < vocab_size:
#         new_word_index[word] = index
#     else:
#         new_word_index[word] = new_index
#         new_index += 1

# # Update tokenizer's word index
# tokenizer.word_index = new_word_index
# tokenizer.index_word = {index: word for word, index in new_word_index.items()}

# # Check if the start token index is now valid
# start_token_index = tokenizer.word_index.get('\t', None)
# if start_token_index is None or start_token_index >= vocab_size:
#     raise ValueError("Start token index is still invalid after re-indexing.")

# Step 3: Convert questions and answers to sequences
question_sequences = tokenizer.texts_to_sequences(questions)
answer_sequences = tokenizer.texts_to_sequences(answers)

# Step 4: Pad sequences to ensure uniform length
max_len_questions = max([len(seq) for seq in question_sequences])
max_len_answers = max([len(seq) for seq in answer_sequences])

encoder_input_data = pad_sequences(question_sequences, maxlen=max_len_questions, padding='post')
decoder_input_data = pad_sequences(answer_sequences, maxlen=max_len_answers, padding='post')

# Step 5: Prepare decoder target data
# The decoder_target_data is ahead of decoder_input_data by one timestep and will not include the start token.
decoder_target_data = []
for seq in answer_sequences:
    decoder_target_data.append(seq[1:])  # Skip the start token

decoder_target_data = pad_sequences(decoder_target_data, maxlen=max_len_answers, padding='post')

# Check the length of encoder_input_data and decoder_target_data
print("Length of encoder_input_data:", len(encoder_input_data))
print("Length of decoder_target_data:", len(decoder_target_data))

# Ensure they have the same length
if len(encoder_input_data) != len(decoder_target_data):
    min_length = min(len(encoder_input_data), len(decoder_target_data))
    encoder_input_data = encoder_input_data[:min_length]
    decoder_target_data = decoder_target_data[:min_length]

print("Length of encoder_input_data:", len(encoder_input_data))
print("Length of decoder_target_data:", len(decoder_target_data))

# Split the data into training and validation sets
validation_split = 0.2
(train_encoder_input_data, val_encoder_input_data,
 train_decoder_input_data, val_decoder_input_data,
 train_decoder_target_data, val_decoder_target_data) = train_test_split(
    encoder_input_data, decoder_input_data, decoder_target_data,
    test_size=validation_split, random_state=42)

# # Now you can fit the model
# # Make sure you also have the decoder_input_data which is required for Seq2Seq model
# model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=64, epochs=100)

# Step 6: One-hot encode the decoder target data
train_decoder_target_data_one_hot = tf.keras.utils.to_categorical(train_decoder_target_data, len(tokenizer.word_index) + 1)
val_decoder_target_data_one_hot = tf.keras.utils.to_categorical(val_decoder_target_data, len(tokenizer.word_index) + 1)

# Now, encoder_input_data, decoder_input_data, and decoder_target_data_one_hot are ready for model training
print(len(train_encoder_input_data), len(val_encoder_input_data),
 len(train_decoder_input_data), len(val_decoder_input_data),
 len(train_decoder_target_data), len(val_decoder_target_data), len(train_decoder_target_data_one_hot), len(val_decoder_target_data_one_hot))

New vocab_size: 76
Length of encoder_input_data: 301
Length of decoder_target_data: 301
Length of encoder_input_data: 301
Length of decoder_target_data: 301
240 61 240 61 240 61 240 61


# **Training Model**

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
#from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Assuming `input_tensor` and `target_tensor` are processed data tensors
# Assuming you have already fitted a tokenizer on your text data
vocab_size = len(tokenizer.word_index) + 1  # +1 for the padding token


# Create a Seq2Seq Model
encoder_inputs = tf.keras.Input(shape=(None,))
encoder_embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=256)(encoder_inputs)
encoder_lstm = tf.keras.layers.LSTM(512, return_state=True, dropout=0.4)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
#encoder_outputs, state_h, state_c = tf.keras.layers.LSTM(1024, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = tf.keras.Input(shape=(None,))
decoder_embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=256)(decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM(512, return_sequences=True, return_state=True, dropout=0.4)
#decoder_lstm = tf.keras.layers.LSTM(1024, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


# Compile & Train the Model
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
# Assuming encoder_input_data, decoder_input_data, and decoder_target_data_one_hot are correctly defined
# model.fit(
#     [encoder_input_data, decoder_input_data],  # Input to the model: encoder and decoder input
#     decoder_target_data_one_hot,              # Target data: One-hot encoded target sequences
#     batch_size=64,
#     epochs=100
# )

# Learning Rate Scheduler or Early Stopping
#early_stopping = EarlyStopping(monitor='val_loss', patience=10)
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0005)

# Create a checkpoint callback
#checkpoint = ModelCheckpoint('model_checkpoint.keras', monitor='val_loss', save_best_only=True)

model.fit(
    [train_encoder_input_data, train_decoder_input_data], 
    train_decoder_target_data_one_hot,
    #callbacks=[checkpoint, early_stopping, reduce_lr],              
    validation_data=([val_encoder_input_data, val_decoder_input_data], val_decoder_target_data_one_hot),
    batch_size=32,
    epochs=200 #using early stopping
)

# Assuming you have a validation set prepared
# Split your data into training and validation sets
# train_encoder_input_data, val_encoder_input_data, train_decoder_input_data, val_decoder_input_data, train_decoder_target_data, val_decoder_target_data

# Save the trained model
model.save('bangla_chatbot_model.keras3')

Epoch 1/200


8/8 [==============================] - 14s 1s/step - loss: 3.2732 - accuracy: 0.5988 - val_loss: 1.8227 - val_accuracy: 0.6702
Epoch 2/200
8/8 [==============================] - 8s 995ms/step - loss: 1.5787 - accuracy: 0.6817 - val_loss: 1.5975 - val_accuracy: 0.6651
Epoch 3/200
8/8 [==============================] - 8s 1s/step - loss: 1.4819 - accuracy: 0.6818 - val_loss: 1.5537 - val_accuracy: 0.6631
Epoch 4/200
8/8 [==============================] - 8s 1s/step - loss: 1.4959 - accuracy: 0.6810 - val_loss: 1.5279 - val_accuracy: 0.6619
Epoch 5/200
8/8 [==============================] - 8s 1s/step - loss: 1.4251 - accuracy: 0.6798 - val_loss: 1.4650 - val_accuracy: 0.6604
Epoch 6/200
8/8 [==============================] - 9s 1s/step - loss: 1.3901 - accuracy: 0.6791 - val_loss: 1.5366 - val_accuracy: 0.6639
Epoch 7/200
8/8 [==============================] - 8s 1s/step - loss: 1.4619 - accuracy: 0.6889 - val_loss: 1.4798 - val_accuracy: 0.6733
Epoch 8/200
8/8 [===========

INFO:tensorflow:Assets written to: bangla_chatbot_model.keras3\assets


# **RESPONSE GENERATOR**

In [5]:
import numpy as np
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model('bangla_chatbot_model.keras2')

# Model Inference Setup
# Encoder Inference
encoder_model = tf.keras.Model(encoder_inputs, encoder_states)

# Decoder Inference
decoder_state_input_h = tf.keras.Input(shape=(512,))
decoder_state_input_c = tf.keras.Input(shape=(512,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = tf.keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)


# Ensure start token is in the tokenizer's word index and its index is valid
start_token = '\t'
if start_token in tokenizer.word_index:
    start_token_index = tokenizer.word_index[start_token]
    if start_token_index >= vocab_size:
        raise ValueError("Start token index exceeds vocabulary size.")
else:
    raise ValueError("Start token not found in tokenizer's word index.")

# Function to generate response

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer.word_index['\t']  # Start token

    stop_condition = False
    decoded_sentence = ''
    iteration_count = 0
    max_iterations = 100  # Set a limit to iterations for debugging


    while not stop_condition and iteration_count < max_iterations:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        print(f"Predicted token index: {sampled_token_index}")  # Debug print


        if sampled_token_index != 0:  # Skip padding token
            sampled_char = tokenizer.index_word.get(sampled_token_index, '?')  # Use '?' for unknown indices
            decoded_sentence += sampled_char

        print(f"Decoded so far: {decoded_sentence}")

        if sampled_char == '\n' or len(decoded_sentence) > max_len_answers:
            stop_condition = True

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

        iteration_count += 1

    return decoded_sentence

# Example usage
# Convert the input question to sequence
input_question = input()
print(input_question)
input_seq = tokenizer.texts_to_sequences([input_question])
input_seq = pad_sequences(input_seq, maxlen=max_len_questions, padding='post')

# Debug tokenization
print("Tokenized sequence:", input_seq)

# # Check what token 54 represents
# print("Token 54 corresponds to:", tokenizer.index_word[54])

# Generate response
response = decode_sequence(input_seq)
print("Response:", response)


আপনি কেমন আছেন?
Tokenized sequence: [[31 12  5  4 55  7  6 13  5 55 31 37  6  5  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
1/1 [==============================] - 0s 310ms/step
Predicted token index: 55
Decoded so far:  
1/1 [==============================] - 0s 19ms/step
Predicted token index: 31
Decoded so far:  আ
1/1 [==============================] - 0s 21ms/step
Predicted token index: 12
Decoded so far:  আপ
1/1 [==============================] - 0s 20ms/step
Predicted token index: 5
Decoded so far:  আপন
1/1 [==============================] - 0s 23ms/step
Predicted token index: 4
Decoded so far:  আপনি
1/1 [==============================] - 0s 20ms/step
Predicted token index: 55
Decoded so far:  আপনি 
1/1 [==============================] - 0s 21ms/step
Predicted token index: 7
Decod

# **FAILED ATTEMPT**

In [ ]:
# def decode_sequence(input_seq):
#     # Encode the input sequence to get the internal state
#     states_value = encoder_model.predict(input_seq)

#     # Generate empty target sequence of length 1 with only the start character
#     target_seq = np.zeros((1, 1))
#     #target_seq[0, 0] = tokenizer.word_index['\t']   # '\t' is the start token for the response
#     target_seq[0, 0] = start_token_index  

#     # Loop for generating response
#     stop_condition = False
#     decoded_sentence = ''
#     while not stop_condition:
#         output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

#         # Sample a token
#         sampled_token_index = np.argmax(output_tokens[0, -1, :])
#         sampled_char = tokenizer.index_word[sampled_token_index]
#         decoded_sentence += sampled_char

#         # Exit condition: either hit max length or find stop token.
#         if sampled_char == '\n' or len(decoded_sentence) > max_len_answers:
#             stop_condition = True

#         # Update the target sequence (length 1).
#         target_seq = np.zeros((1, 1))
#         target_seq[0, 0] = sampled_token_index

#         # Update internal states
#         states_value = [h, c]

#     return decoded_sentence

# def decode_sequence(input_seq):
#     # Encode the input sequence to get the internal state
#     states_value = encoder_model.predict(input_seq)

#     # Generate empty target sequence of length 1 with only the start character
#     target_seq = np.zeros((1, 1))
#     target_seq[0, 0] = tokenizer.word_index['\t']  # '\t' is the start token for the response

#     # Loop for generating response
#     stop_condition = False
#     decoded_sentence = ''
#     while not stop_condition:
#         output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

#         # Sample a token
#         sampled_token_index = np.argmax(output_tokens[0, -1, :])

#         # Check if the sampled token index is 0 (padding token)
#         if sampled_token_index != 0:
#             sampled_char = tokenizer.index_word[sampled_token_index]
#             decoded_sentence += sampled_char

#         # Exit condition: either hit max length or find stop token.
#         if sampled_char == '\n' or len(decoded_sentence) > max_len_answers:
#             stop_condition = True

#         # Update the target sequence (length 1).
#         target_seq = np.zeros((1, 1))
#         target_seq[0, 0] = sampled_token_index

#         # Update internal states
#         states_value = [h, c]

#     return decoded_sentence

# def decode_sequence(input_seq):
#     states_value = encoder_model.predict(input_seq)
#     target_seq = np.zeros((1, 1))
#     target_seq[0, 0] = tokenizer.word_index['\t']  # Start token

#     stop_condition = False
#     decoded_sentence = ''
#     sampled_char = ''  # Initialize sampled_char

#     while not stop_condition:
#         output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
#         sampled_token_index = np.argmax(output_tokens[0, -1, :])

#         if sampled_token_index != 0:  # Skip padding token
#             sampled_char = tokenizer.index_word[sampled_token_index]
#             decoded_sentence += sampled_char

#         print(f"Decoded so far: {decoded_sentence}")  # Debug print

#         if sampled_char == '\n' or len(decoded_sentence) > max_len_answers:
#             stop_condition = True

#         target_seq = np.zeros((1, 1))
#         target_seq[0, 0] = sampled_token_index
#         states_value = [h, c]

#     return decoded_sentence

# I noticed one thing: every time I retrain the model, the model accuracy goes down a bit. When I first trained the model, the accuracy was 0.8756, and now, after retraining the model a fourth time, the accuracy has gone down to 0.8474. Does this mean the model is not trained properly, and is this the reason that, while generating responses, the model entered an infinite loop?
# Should I provide you with the whole code, from preprocessing datasets to training the model and generating responses, so that you can analyze the code and help me debug it?